In [ ]:
import numpy as np


# 在这个方法中编写任何的初始化逻辑。context对象将会在你的算法策略的任何方法之间做传递。
def init(context):
    # 在context中保存全局变量
    context.quantile = 5

    scheduler.run_monthly(factor_rollback, tradingday=1)


def factor_rollback(context, bar_dict):
    # 选出因子数据，选出不同组的位置进行回测
    q = query(fundamentals.financial_indicator.earnings_per_share)

    fund = get_fundamentals(q)

    # 进行数据处理
    # 变成行是股票、列是指标的数据
    factor = fund.T

    # 去极值、标准化
    factor['earnings_per_share'] = mad(factor['earnings_per_share'])
    factor['earnings_per_share'] = stand(factor['earnings_per_share'])

    # 求分位数是对于一列数据，dataframe求不了
    data = factor.iloc[:, 0]

    # 按照分位去分成5组，分别进行回测
    # quantie(0.2)
    # [0.2, 0.4, 0.6, 0.8]
    if context.quantile == 1:
        data = data[data <= data.quantile(0.2)]
    elif context.quantile == 2:
        data = data[(data > data.quantile(0.2)) & (data <= data.quantile(0.4))]
    elif context.quantile == 3:
        data = data[(data > data.quantile(0.4)) & (data <= data.quantile(0.6))]
    elif context.quantile == 4:
        data = data[(data > data.quantile(0.6)) & (data <= data.quantile(0.8))]
    elif context.quantile == 5:
        data = data[data > data.quantile(0.8)]

    # 建立回测的股票池
    context.stock_list = data.index

    # 卖
    for stock in context.portfolio.positions.keys():

        # 判断股票仓位是否>0
        if context.portfolio.positions[stock].quantity > 0:

            if stock not in context.stock_list:
                order_target_percent(stock, 0)

    weight = 1.0 / len(context.stock_list)
    # 卖
    for stock in context.stock_list:
        order_target_percent(stock, weight)


# before_trading此函数会在每天策略交易开始前被调用，当天只会被调用一次
def before_trading(context):
    pass


# 你选择的证券的数据更新将会触发此段逻辑，例如日或分钟历史数据切片或者是实时数据切片更新
def handle_bar(context, bar_dict):
    # 开始编写你的主要的算法逻辑
    pass


# after_trading函数会在每天交易结束后被调用，当天只会被调用一次
def after_trading(context):
    pass


def mad(factor):
    """3倍中位数去极值
    """
    # 求出因子值的中位数
    med = np.median(factor)

    # 求出因子值与中位数的差值，进行绝对值
    mad = np.median(np.abs(factor - med))

    # 定义几倍的中位数上下限
    high = med + (3 * 1.4826 * mad)
    low = med - (3 * 1.4826 * mad)

    # 替换上下限以外的值
    factor = np.where(factor > high, high, factor)
    factor = np.where(factor < low, low, factor)
    return factor


def stand(factor):
    """自实现标准化
    """
    mean = factor.mean()
    std = factor.std()
    return (factor - mean) / std